In [1]:
from scipy.interpolate import lagrange
import numpy as np
import random
from random import randint
from numpy.polynomial.polynomial import Polynomial as Poly
import numpy.polynomial.polynomial as polynomial
import pandas as pd

In [2]:
data = [[2, 4], [3, 5], [5, 7] , [7, 10 ] , [9, 15]] 
  
# Create a dataframe from the list data
df = pd.DataFrame(data, columns = ['Age', 'Height']) 
  
# print the dataframe. 
df 

,Age,Height
0,2,4
1,3,5
2,5,7
3,7,10
4,9,15


In [3]:
# initialize variables
degree = 4
N = 10
P = 41
SHARE_POINTS = [ p for p in range(1, N+1) ]
LAGRANGE_POINTS = [ l for l in range(1, 2*degree+1) ]

In [4]:
# calculates the lagrange basis polynomial coefficients
def lagrange_coefficients(points):
    def P(x):
        l_coefficients = []
        n = len(points)
        for i in range(n):
            xi = points[i]

            def g(i, n):

                tot_mul = 1
                for j in range(n):
                    if i == j:
                        continue
                    xj = points[j]
                    tot_mul *= (x - xj) / float(xi - xj)

                return tot_mul 

            l_coefficients.append( round(g(i, n), 1 ) )
        return l_coefficients
    return P

In [5]:
# Given the list of shares named "numbers", calcultes the sum 
def addition(numbers):
    total = 0
    arrays_of_numbers = [ np.array(num)[:degree] for num in numbers ]
    for item in arrays_of_numbers:
        total += item
    z = Share()
    z.shares = list(total)
    z.degree = degree
    return z
    
# Given a list of shares named numbers, calculates the subtraction
def subtraction(numbers):
    
    arrays_of_numbers = [ np.array(num)[:degree] for num in numbers ]
    arrays_of_numbers = np.array(arrays_of_numbers)
    arrays_of_numbers = arrays_of_numbers.astype(np.float)
    total = arrays_of_numbers[0]
    for count,item in enumerate(arrays_of_numbers):
        if count == 0:
            continue
        total -= item
    
        
    z = Share()
    z.shares = list(total)
    z.degree = degree
    return z

def multiplication(x,y):

    b = lagrange_coefficients(LAGRANGE_POINTS)(0)
    z = Share()
    z.degree = 2*degree
    x = Share(reconstruct_polynomial(x))
    y = Share(reconstruct_polynomial(y))
    xShares = np.array(x.shares)[:z.degree]
    yShares = np.array(y.shares)[:z.degree]
    total = xShares * yShares * b
    z = Share(round(sum(total),1))
    return z

def division(x,y):
    x = Share(reconstruct_polynomial(x))
    y = Share(1/reconstruct_polynomial(y))
    z = Share()
    z.degree = 2*degree
    xShares = np.array(x.shares)[:z.degree]
    yShares = np.array(y.shares)[:z.degree]
    total = xShares * yShares
    z.shares = list(total)
    return z

In [6]:
class Share():
    
    def __init__(self, secret=None):
        self.shares = create_shares(secret) if secret is not None else []
        self.degree = degree
        self.secret = secret

In [7]:
def create_polynomial(secret):
    coefficients = [randint(1, P-1) for coef in range(degree)]
    coefficients[-1] = secret
    polynomial = np.poly1d(coefficients)
    return polynomial

In [8]:
def reconstruct_polynomial(z):
    points = [i for i in range(1, z.degree+1) ]
    poly = lagrange(points, z.shares)
    return round(poly(0),2)


In [9]:
def create_shares(secret):
    #create the polynomial
    polynomial = create_polynomial(secret)
    shares = [ polynomial(i) for i in SHARE_POINTS]
    return shares

In [10]:
# calculate the sum of all ages
AllAges = df.loc[:,'Age']
numbers = Share()
numbers = [ Share(num).shares for num in list(AllAges) ]
sumAge = addition(numbers)

In [11]:
reconstruct_polynomial(sumAge)

26.0

In [12]:
#calclulate the sum of all heights
AllHeights = df.loc[:,'Height']
numbers = Share()
numbers = [ Share(num).shares for num in list(AllHeights) ]
sumHeights = addition(numbers)

In [13]:
reconstruct_polynomial(sumHeights)

41.0

In [14]:
# calculate the square of each age
Ages = df.loc[:, "Age"]
AgeShares = [Share(age) for age in Ages]
squareAges = [multiplication(share,share) for share in AgeShares]

In [15]:
# calculate the square of each height
Heights = df.loc[:, "Height"]
HeightShares = [Share(height) for height in Heights]
squareHeights = [multiplication(share,share) for share in HeightShares]

In [16]:
# product of Age and Height
xy = [ multiplication(age, height) for age, height in zip(AgeShares, HeightShares) ]

In [17]:
# sum of squares of Ages
squareAgesShares = [age.shares for age in squareAges]
SumSquareAges = addition(squareAgesShares)

In [18]:
reconstruct_polynomial(SumSquareAges)

168.0

In [19]:
# sum of the (product of Age and Height)
xyShares = [ num.shares for num in xy ]
sumXY = addition(xyShares)


In [20]:
reconstruct_polynomial(sumXY)

263.0

In [21]:
# total number of rows of the dataset
numRows = Share(len(df.index))

In [22]:
reconstruct_polynomial(numRows)

5.0

In [23]:
#calculate the slope m
# m = var1 - var2 / var3 - var4
var1 = multiplication(sumXY, numRows)
var2 = multiplication(sumHeights,sumAge)
var3 = multiplication(SumSquareAges, numRows)
var4 = multiplication(sumAge, sumAge)

var12 = subtraction([var1.shares,var2.shares])
var34 = subtraction([var3.shares, var4.shares])
m = division(var12,var34)

In [24]:
reconstruct_polynomial(m)

1.52

In [30]:
#calculate the intercept b
# b = (sumHeights - m * sumAge) / numRowsa
var5 = multiplication(m, sumAge)
var6 = subtraction([sumHeights.shares, var5.shares])
b = division(var6, numRows)
print(reconstruct_polynomial(b))

0.3


In [39]:
# the formula of linear regression is 
# y = m*x + b 
# where x is the input age
myAge = Share(8)
mx = multiplication(m, myAge)
res = addition([mx.shares, b.shares])

In [40]:
reconstruct_polynomial(res)

12.5